## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/3', creation_time=1718881439466, experiment_id='3', last_update_time=1718881439466, lifecycle_stage='active', name='my-cool-experiment2', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1718877908629, experiment_id='2', last_update_time=1718877908629, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1718809930065, experiment_id='1', last_update_time=1718809930065, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1718808575988, experiment_id='0', last_update_time=1718808575988, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/06/20 11:54:12 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'mlflow-artifacts:/4/8ba45c5b22f949499db686e0868f29af/artifacts'


In [ ]:
mlflow.list_experiments()

### Interacting with the model registry

In [5]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [18]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1718884637676, description='', last_updated_timestamp=1718884681896, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1718884681896, current_stage='None', description='', last_updated_timestamp=1718884681896, name='iris-classifier', run_id='8ba45c5b22f949499db686e0868f29af', run_link='', source='mlflow-artifacts:/4/8ba45c5b22f949499db686e0868f29af/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>], name='iris-classifier', tags={}>,
 <RegisteredModel: aliases={'Beta': '4', 'Production': '4', 'Sigma': '2', 'beta': '9'}, creation_timestamp=1718877186169, description='', last_updated_timestamp=1718883217659, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1718883217659, current_stage='None', description='', last_updated_timestamp=1718883217659, name='nyc_taxi_model', run_id='f5520aca4b114fb199caa353f566ab35', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-trac

In [6]:
client.get_registered_model("iris-classifier")

RestException: RESOURCE_DOES_NOT_EXIST: Registered Model with name=iris-classifier not found

In [17]:
run_id = client.search_runs(experiment_ids=["4"])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2024/06/20 11:58:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 2
Created version '2' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1718884681896, current_stage='None', description='', last_updated_timestamp=1718884681896, name='iris-classifier', run_id='8ba45c5b22f949499db686e0868f29af', run_link='', source='mlflow-artifacts:/4/8ba45c5b22f949499db686e0868f29af/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>